In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train,y_train),(x_test,y_test)= fashion_mnist.load_data()


In [3]:
x_train, y_train = x_train[6000:], y_train[6000:]
x_valid, y_valid = x_train[:6000], y_train[:6000]
len_train = len(x_train)
len_valid = len(x_valid)
print(len_train,len_valid)

54000 6000


In [4]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [5]:

y_train[:10]

array([8, 6, 4, 4, 6, 5, 5, 8, 7, 3], dtype=uint8)

In [7]:
train_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, 
                                                                rotation_range=45,
                                                                width_shift_range=.15,
                                                                height_shift_range=.15,
                                                                horizontal_flip=True,
                                                                zoom_range=0.5)
train_generator.fit(x_train)
train_gen = train_generator.flow(x_train,y_train,
                     shuffle=True,
                    )

test_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator.fit(x_test)
test_gen = test_generator.flow(x_test,y_test)

valid_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_generator.fit(x_valid)
valid_gen = valid_generator.flow(x_valid,y_valid)
                    

In [9]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, 3, padding='same', input_shape=(28,28,1), activation = 'relu'),
    keras.layers.Conv2D(32, 3,padding='same', activation = 'relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Dropout(0),#0代表随机比例
    keras.layers.Conv2D(64, 3, padding='same',  activation = 'relu'),
    keras.layers.Conv2D(64, 3,padding='same', activation = 'relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Dropout(0),
    keras.layers.Conv2D(128, 3, padding='same', activation = 'relu'),
    keras.layers.Conv2D(128, 3,padding='same', activation = 'relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Dropout(0),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10)
    ])
model.compile(loss =keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     optimizer = 'adam',
                     metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0

In [10]:
callbacks = [keras.callbacks.EarlyStopping(patience=5,min_delta=0.01,)]
history = model.fit_generator(train_gen, epochs = 5, steps_per_epoch=54000//32,validation_data = valid_gen,validation_steps = 6000//32)



Epoch 1/5
  38/1687 [..............................] - ETA: 18:59 - loss: 2.1283 - accuracy: 0.2031

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_gen, verbose=2)

In [ ]:
pd.DataFrame(history.history).plot()